In [35]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [36]:
base_votos = pd.read_csv("votacoesVotos-2021 (1).csv",sep=';')
base_votacao = pd.read_csv("votacoesObjetos-2021.csv",sep=';')

In [37]:
base_votacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49338 entries, 0 to 49337
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idVotacao             49338 non-null  object 
 1   uriVotacao            49338 non-null  object 
 2   data                  49338 non-null  object 
 3   descricao             49338 non-null  object 
 4   proposicao_id         49338 non-null  int64  
 5   proposicao_uri        49338 non-null  object 
 6   proposicao_ementa     48400 non-null  object 
 7   proposicao_codTipo    49338 non-null  int64  
 8   proposicao_siglaTipo  49338 non-null  object 
 9   proposicao_numero     49338 non-null  int64  
 10  proposicao_ano        14030 non-null  float64
 11  proposicao_titulo     49338 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 4.5+ MB


In [38]:
x = base_votos.drop(['uriVotacao','dataHoraVoto','uriVotacao','deputado_uri'
                   ,'deputado_uriPartido','deputado_siglaUf'
                   ,'deputado_idLegislatura','deputado_urlFoto'],axis=1)

In [39]:
print(x.voto.value_counts())

x.replace({"voto":{"Sim" : 1, "Não" : 0, "Obstrução" : 0, "Abstenção": 0.5, "Artigo 17": 0.5}}, inplace = True)

x = x.pivot_table('voto', ['deputado_id','deputado_nome','deputado_siglaPartido'], 'idVotacao')

Sim          159395
Não          133770
Obstrução      2741
Abstenção       661
Artigo 17       647
Name: voto, dtype: int64


In [40]:
propostas_selecionadas = [#'2282791-60',
                          '2293449-187',
                          '2275537-117',
                          '2275537-102',
                          '2165194-130',
                          '2210757-104',
                          '2210757-93',
                          '531331-317',
                          '531331-284',
                          '2166595-59']


In [41]:
base_votacao_selecionadas = base_votacao[base_votacao['idVotacao'].isin(propostas_selecionadas)]

In [42]:
base_votacao_selecionadas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 23630 to 46740
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idVotacao             60 non-null     object 
 1   uriVotacao            60 non-null     object 
 2   data                  60 non-null     object 
 3   descricao             60 non-null     object 
 4   proposicao_id         60 non-null     int64  
 5   proposicao_uri        60 non-null     object 
 6   proposicao_ementa     60 non-null     object 
 7   proposicao_codTipo    60 non-null     int64  
 8   proposicao_siglaTipo  60 non-null     object 
 9   proposicao_numero     60 non-null     int64  
 10  proposicao_ano        28 non-null     float64
 11  proposicao_titulo     60 non-null     object 
dtypes: float64(1), int64(3), object(8)
memory usage: 6.1+ KB


In [43]:
x1 = x[propostas_selecionadas]

In [44]:
x1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 553 entries, (62881, 'Danilo Forte', 'PSDB') to (218086, 'Ely Santos', 'REPUBLICANOS')
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   2293449-187  479 non-null    float64
 1   2275537-117  444 non-null    float64
 2   2275537-102  429 non-null    float64
 3   2165194-130  459 non-null    float64
 4   2210757-104  441 non-null    float64
 5   2210757-93   442 non-null    float64
 6   531331-317   440 non-null    float64
 7   531331-284   429 non-null    float64
 8   2166595-59   427 non-null    float64
dtypes: float64(9)
memory usage: 50.4+ KB


In [45]:
x1 = x1.dropna()

In [46]:
x1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 211 entries, (62881, 'Danilo Forte', 'PSDB') to (215045, 'Pedro Augusto Palareti', 'PSD')
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   2293449-187  211 non-null    float64
 1   2275537-117  211 non-null    float64
 2   2275537-102  211 non-null    float64
 3   2165194-130  211 non-null    float64
 4   2210757-104  211 non-null    float64
 5   2210757-93   211 non-null    float64
 6   531331-317   211 non-null    float64
 7   531331-284   211 non-null    float64
 8   2166595-59   211 non-null    float64
dtypes: float64(9)
memory usage: 24.7+ KB


In [47]:
from sklearn.cluster import KMeans

In [48]:
kmeans = KMeans(init="random", n_clusters=3, n_init=10,max_iter=300)

In [49]:
labels = kmeans.fit(x1)
prediction = kmeans.predict(x1)
prediction = pd.DataFrame(prediction)

In [50]:
x1 = x1.reset_index(level =['deputado_id','deputado_nome','deputado_siglaPartido'])
x1['Cluster'] = prediction

In [51]:
x1.head()

idVotacao  deputado_id      deputado_nome deputado_siglaPartido  2293449-187  \
0                62881       Danilo Forte                  PSDB          0.0   
1                66828      Fausto Pinato                    PP          1.0   
2                73433  Arlindo Chinaglia                    PT          0.0   
3                73466       Rubens Bueno             CIDADANIA          1.0   
4                73482   Henrique Fontana                    PT          0.0   

idVotacao  2275537-117  2275537-102  2165194-130  2210757-104  2210757-93  \
0                  0.0          0.0          1.0          1.0         1.0   
1                  0.0          0.0          1.0          1.0         1.0   
2                  0.0          0.0          1.0          1.0         1.0   
3                  1.0          1.0          1.0          1.0         1.0   
4                  0.0          0.0          1.0          1.0         1.0   

idVotacao  531331-317  531331-284  2166595-59  Cluster  
0                 1.0         0.0         1.0        1  
1                 1.0         0.0         1.0        1  
2                 1.0         0.0         1.0        1  
3                 0.0         0.0         1.0        2  
4                 1.0         0.0         1.0        1

In [52]:
cluster = x1.groupby(['deputado_siglaPartido','Cluster']).size().reset_index(name='counts')

In [53]:
cluster.head()

deputado_siglaPartido  Cluster  counts
0                AVANTE        0       1
1                AVANTE        1       1
2                AVANTE        2       1
3             CIDADANIA        0       1
4             CIDADANIA        1       1

In [54]:
cluster_final = cluster.pivot_table('counts', ['deputado_siglaPartido'], 'Cluster')


In [55]:
cluster_final

Cluster                  0     1     2
deputado_siglaPartido                 
AVANTE                 1.0   1.0   1.0
CIDADANIA              1.0   1.0   4.0
DEM                    NaN   2.0   4.0
MDB                    NaN   4.0   5.0
NOVO                   8.0   NaN   NaN
PCdoB                  NaN   5.0   NaN
PDT                    4.0   1.0   2.0
PL                     NaN   8.0   3.0
PODE                   1.0   2.0   2.0
PP                     NaN   6.0   2.0
PROS                   NaN   1.0   4.0
PSB                    2.0   3.0  11.0
PSC                    NaN   5.0   1.0
PSD                    NaN   8.0   5.0
PSDB                   2.0   4.0   7.0
PSL                    3.0  22.0  10.0
PSOL                   3.0   NaN   NaN
PT                     NaN  32.0   NaN
PTB                    NaN   NaN   1.0
PV                     NaN   NaN   1.0
REPUBLICANOS           NaN  13.0   1.0
SOLIDARIEDADE          NaN   3.0   1.0

In [56]:
x1.to_csv("clusters_deputado_siglaPartido.csv", sep=';')

In [57]:
x1

idVotacao  deputado_id           deputado_nome deputado_siglaPartido  \
0                62881            Danilo Forte                  PSDB   
1                66828           Fausto Pinato                    PP   
2                73433       Arlindo Chinaglia                    PT   
3                73466            Rubens Bueno             CIDADANIA   
4                73482        Henrique Fontana                    PT   
..                 ...                     ...                   ...   
206             213762           Carla Dickson                  PROS   
207             215042        Ricardo da Karol                   PSC   
208             215043            Josivaldo JP                  PODE   
209             215044               Vivi Reis                  PSOL   
210             215045  Pedro Augusto Palareti                   PSD   

idVotacao  2293449-187  2275537-117  2275537-102  2165194-130  2210757-104  \
0                  0.0          0.0          0.0          1.0          1.0   
1                  1.0          0.0          0.0          1.0          1.0   
2                  0.0          0.0          0.0          1.0          1.0   
3                  1.0          1.0          1.0          1.0          1.0   
4                  0.0          0.0          0.0          1.0          1.0   
..                 ...          ...          ...          ...          ...   
206                1.0          1.0          0.0          1.0          1.0   
207                1.0          0.0          0.0          1.0          1.0   
208                1.0          0.0          1.0          1.0          1.0   
209                0.0          1.0          1.0          1.0          1.0   
210                1.0          0.0          0.0          1.0          1.0   

idVotacao  2210757-93  531331-317  531331-284  2166595-59  Cluster  
0                 1.0         1.0         0.0         1.0        1  
1                 1.0         1.0         0.0         1.0        1  
2                 1.0         1.0         0.0         1.0        1  
3                 1.0         0.0         0.0         1.0        2  
4                 1.0         1.0         0.0         1.0        1  
..                ...         ...         ...         ...      ...  
206               1.0         1.0         0.0         1.0        2  
207               1.0         1.0         0.0         1.0        1  
208               1.0         1.0         1.0         1.0        2  
209               1.0         0.0         0.0         0.0        0  
210               1.0         1.0         0.0         1.0        1  

[211 rows x 13 columns]